In [1]:
from huggingface_hub import login
import wandb
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers import SentenceTransformerModelCardData
from sentence_transformers.util import cos_sim
import json
from datasets import load_dataset, concatenate_datasets
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

2025-05-21 09:43:09.021062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747820589.502239      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747820589.623382      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


> read dataset

In [2]:
df_legal=pd.read_csv("/kaggle/input/data-embedding-legal-final-kltn/final_train_1.csv")


In [3]:
df_legal.shape

(71714, 4)

> Select question and context columns

In [4]:
df_legal=df_legal[['question','context']]

> rename 

In [5]:
df_legal = df_legal.rename(columns={'question': 'anchor', 'context': 'positive'})

> Create column id

In [6]:
df_legal["id"] = range(len(df_legal))

> Prepare train,test,val

In [7]:
train_df_legal, temp_df_legal = train_test_split(df_legal, test_size=0.2, random_state=42)

# Chia temp thành tập test và val
test_df_legal, val_df_legal = train_test_split(temp_df_legal, test_size=0.5, random_state=42)

train_df_legal.to_json("/kaggle/working/train_dataset_legal.json", orient="records", lines=True)
test_df_legal.to_json("/kaggle/working/test_dataset_legal.json", orient="records", lines=True)
val_df_legal.to_json("/kaggle/working/val_dataset_legal.json", orient="records", lines=True)

> reload dataset

In [8]:
test_dataset_legal = load_dataset("json", data_files="/kaggle/working/test_dataset_legal.json", split="train")
val_dataset_legal=load_dataset("json",data_files='/kaggle/working/val_dataset_legal.json',split='train')
train_dataset_legal = load_dataset("json", data_files="/kaggle/working/train_dataset_legal.json", split="train")
corpus_dataset_legal = concatenate_datasets([train_dataset_legal,val_dataset_legal,test_dataset_legal])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

## 1. hiieu/halong_embedding

> Load Model Embedding Pre-train

In [9]:
model_id = "hiieu/halong_embedding"  # Hugging Face model ID
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

> Prepare dataset for evaluation

In [10]:
 # Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset_legal["id"], corpus_dataset_legal["positive"])
)  # Our corpus (cid => document)
queries = dict(
    zip(test_dataset_legal["id"], test_dataset_legal["anchor"])
)  # Our queries (qid => question)
 
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for q_id in queries:
    relevant_docs[q_id] = [q_id]

In [11]:
matryoshka_evaluators = []
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        mrr_at_k=[1, 3, 5, 10, 20,25],
        ndcg_at_k=[1, 3, 5, 10, 20,25],
        precision_recall_at_k=[1, 3, 5, 10, 20,25],
        accuracy_at_k=[1,3,5,10,20,25],
        map_at_k=[10, 100],
        truncate_dim=dim,                  # <-- cắt xuống dim
        name=f"dim_{dim}", # tên riêng
        show_progress_bar=True,
        write_csv=False,
        score_functions={"cosine": cos_sim}
    )
    matryoshka_evaluators.append(ir_evaluator)

evaluator = SequentialEvaluator(matryoshka_evaluators)

> Evaluate the model

In [12]:
results = evaluator(model)

target_dims = ["dim_768"]
target_ks = ["1", "3", "5", "10", "20", "25"]

for key, value in results.items():
    if any(dim in key for dim in target_dims) and \
       any(f"@{k}" in key for k in target_ks):
        print(f"{key}: {value}")

# Xuất kết quả ra file JSON
with open("halong_embedding.json", "w") as outfile:
    json.dump(results, outfile, indent=4)
print("Kết quả đã được xuất ra")

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [27:31<00:00, 825.88s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [27:45<00:00, 832.81s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [27:45<00:00, 832.85s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [27:43<00:00, 831.85s/it]


dim_768_cosine_accuracy@1: 0.20568958304281132
dim_768_cosine_accuracy@3: 0.3820945474829173
dim_768_cosine_accuracy@5: 0.46199972109887044
dim_768_cosine_accuracy@10: 0.5999163296611352
dim_768_cosine_accuracy@20: 0.6936271091897922
dim_768_cosine_accuracy@25: 0.7202621670617766
dim_768_cosine_precision@1: 0.20568958304281132
dim_768_cosine_precision@3: 0.12736484916097243
dim_768_cosine_precision@5: 0.09239994421977407
dim_768_cosine_precision@10: 0.05999163296611351
dim_768_cosine_precision@20: 0.03468135545948961
dim_768_cosine_precision@25: 0.028810486682471062
dim_768_cosine_recall@1: 0.20568958304281132
dim_768_cosine_recall@3: 0.3820945474829173
dim_768_cosine_recall@5: 0.46199972109887044
dim_768_cosine_recall@10: 0.5999163296611352
dim_768_cosine_recall@20: 0.6936271091897922
dim_768_cosine_recall@25: 0.7202621670617766
dim_768_cosine_ndcg@1: 0.20568958304281132
dim_768_cosine_ndcg@3: 0.3072570882184224
dim_768_cosine_ndcg@5: 0.34018534038997633
dim_768_cosine_ndcg@10: 0.3844

## 2.VoVanPhuc/sup-SimCSE-VietNamese-phobert-base

> Compute task retrieval of model-pretrained

In [13]:
model_id = "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"  # Hugging Face model ID
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [14]:
results = evaluator(model)

target_dims = ["dim_768"]
target_ks = ["1", "3", "5", "10", "20", "25"]

for key, value in results.items():
    if any(dim in key for dim in target_dims) and \
       any(f"@{k}" in key for k in target_ks):
        print(f"{key}: {value}")

# Xuất kết quả ra file JSON
with open("sup-SimCSE-VietNamese-phobert-base.json", "w") as outfile:
    json.dump(results, outfile, indent=4)

print("Kết quả đã được xuất ra")

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:55<00:00, 447.65s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.14s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:55<00:00, 447.89s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.49s/it]


dim_768_cosine_accuracy@1: 0.10877144052433413
dim_768_cosine_accuracy@3: 0.23106958583182263
dim_768_cosine_accuracy@5: 0.302328824431739
dim_768_cosine_accuracy@10: 0.4413610375122019
dim_768_cosine_accuracy@20: 0.5497141263422117
dim_768_cosine_accuracy@25: 0.5774647887323944
dim_768_cosine_precision@1: 0.10877144052433413
dim_768_cosine_precision@3: 0.07702319527727419
dim_768_cosine_precision@5: 0.06046576488634779
dim_768_cosine_precision@10: 0.04413610375122019
dim_768_cosine_precision@20: 0.027485706317110586
dim_768_cosine_precision@25: 0.023098591549295774
dim_768_cosine_recall@1: 0.10877144052433413
dim_768_cosine_recall@3: 0.23106958583182263
dim_768_cosine_recall@5: 0.302328824431739
dim_768_cosine_recall@10: 0.4413610375122019
dim_768_cosine_recall@20: 0.5497141263422117
dim_768_cosine_recall@25: 0.5774647887323944
dim_768_cosine_ndcg@1: 0.10877144052433413
dim_768_cosine_ndcg@3: 0.17945130823655012
dim_768_cosine_ndcg@5: 0.20868040386129122
dim_768_cosine_ndcg@10: 0.2534

## 3.keepitreal/vietnamese-sbert

In [15]:
model_id = "keepitreal/vietnamese-sbert"  # Hugging Face model ID
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
results = evaluator(model)

target_dims = ["dim_768"]
target_ks = ["1", "3", "5", "10", "20", "25"]

for key, value in results.items():
    if any(dim in key for dim in target_dims) and \
       any(f"@{k}" in key for k in target_ks):
        print(f"{key}: {value}")

# Xuất kết quả ra file JSON
with open("vietnamese-sbert.json", "w") as outfile:
    json.dump(results, outfile, indent=4)

print("Kết quả đã được xuất ra")

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.23s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:57<00:00, 448.68s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:55<00:00, 447.96s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:57<00:00, 448.59s/it]


dim_768_cosine_accuracy@1: 0.13289638823037234
dim_768_cosine_accuracy@3: 0.2751359643006554
dim_768_cosine_accuracy@5: 0.3447217961232743
dim_768_cosine_accuracy@10: 0.4854274159810347
dim_768_cosine_accuracy@20: 0.5941988565053689
dim_768_cosine_accuracy@25: 0.624877980755822
dim_768_cosine_precision@1: 0.13289638823037234
dim_768_cosine_precision@3: 0.09171198810021847
dim_768_cosine_precision@5: 0.06894435922465487
dim_768_cosine_precision@10: 0.04854274159810348
dim_768_cosine_precision@20: 0.029709942825268443
dim_768_cosine_precision@25: 0.024995119230232885
dim_768_cosine_recall@1: 0.13289638823037234
dim_768_cosine_recall@3: 0.2751359643006554
dim_768_cosine_recall@5: 0.3447217961232743
dim_768_cosine_recall@10: 0.4854274159810347
dim_768_cosine_recall@20: 0.5941988565053689
dim_768_cosine_recall@25: 0.624877980755822
dim_768_cosine_ndcg@1: 0.13289638823037234
dim_768_cosine_ndcg@3: 0.2159205409745629
dim_768_cosine_ndcg@5: 0.24460005443874158
dim_768_cosine_ndcg@10: 0.2896195

## 4.bkai-foundation-models/vietnamese-bi-encoder

In [17]:
model_id = "bkai-foundation-models/vietnamese-bi-encoder"  # Hugging Face model ID
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [18]:
results = evaluator(model)

target_dims = ["dim_768"]
target_ks = ["1", "3", "5", "10", "20", "25"]

for key, value in results.items():
    if any(dim in key for dim in target_dims) and \
       any(f"@{k}" in key for k in target_ks):
        print(f"{key}: {value}")

# Xuất kết quả ra file JSON
with open("vietnamese-bi-encoder.json", "w") as outfile:
    json.dump(results, outfile, indent=4)

print("Kết quả đã được xuất ra")

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.35s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:57<00:00, 448.75s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.18s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.29s/it]


dim_768_cosine_accuracy@1: 0.17319760145028587
dim_768_cosine_accuracy@3: 0.35392553339841026
dim_768_cosine_accuracy@5: 0.43912982847580534
dim_768_cosine_accuracy@10: 0.6272486403569935
dim_768_cosine_accuracy@20: 0.7258401896527681
dim_768_cosine_accuracy@25: 0.7498256867940315
dim_768_cosine_precision@1: 0.17319760145028587
dim_768_cosine_precision@3: 0.11797517779947009
dim_768_cosine_precision@5: 0.08782596569516107
dim_768_cosine_precision@10: 0.06272486403569935
dim_768_cosine_precision@20: 0.0362920094826384
dim_768_cosine_precision@25: 0.029993027471761264
dim_768_cosine_recall@1: 0.17319760145028587
dim_768_cosine_recall@3: 0.35392553339841026
dim_768_cosine_recall@5: 0.43912982847580534
dim_768_cosine_recall@10: 0.6272486403569935
dim_768_cosine_recall@20: 0.7258401896527681
dim_768_cosine_recall@25: 0.7498256867940315
dim_768_cosine_ndcg@1: 0.17319760145028587
dim_768_cosine_ndcg@3: 0.276415359993628
dim_768_cosine_ndcg@5: 0.31155859584215323
dim_768_cosine_ndcg@10: 0.3716

## 5.huyydangg/DEk21_hcmute_embedding

In [19]:
model_id = "huyydangg/DEk21_hcmute_embedding"  # Hugging Face model ID
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
) 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [20]:
results = evaluator(model)

target_dims = ["dim_768"]
target_ks = ["1", "3", "5", "10", "20", "25"]

for key, value in results.items():
    if any(dim in key for dim in target_dims) and \
       any(f"@{k}" in key for k in target_ks):
        print(f"{key}: {value}")

 
# Xuất kết quả ra file JSON
with open("DEk21_hcmute_embedding.json", "w") as outfile:
    json.dump(results, outfile, indent=4)

print("Kết quả đã được xuất ra")

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:54<00:00, 447.42s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.08s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:55<00:00, 447.99s/it]


Batches:   0%|          | 0/225 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 2/2 [14:56<00:00, 448.26s/it]


dim_768_cosine_accuracy@1: 0.34165388369822897
dim_768_cosine_accuracy@3: 0.4533537860828336
dim_768_cosine_accuracy@5: 0.5354901687351834
dim_768_cosine_accuracy@10: 0.7060382094547483
dim_768_cosine_accuracy@20: 0.8063031655278204
dim_768_cosine_accuracy@25: 0.8258262445962906
dim_768_cosine_precision@1: 0.34165388369822897
dim_768_cosine_precision@3: 0.1511179286942779
dim_768_cosine_precision@5: 0.10709803374703668
dim_768_cosine_precision@10: 0.07060382094547484
dim_768_cosine_precision@20: 0.040315158276391025
dim_768_cosine_precision@25: 0.033033049783851624
dim_768_cosine_recall@1: 0.34165388369822897
dim_768_cosine_recall@3: 0.4533537860828336
dim_768_cosine_recall@5: 0.5354901687351834
dim_768_cosine_recall@10: 0.7060382094547483
dim_768_cosine_recall@20: 0.8063031655278204
dim_768_cosine_recall@25: 0.8258262445962906
dim_768_cosine_ndcg@1: 0.34165388369822897
dim_768_cosine_ndcg@3: 0.40623126791342307
dim_768_cosine_ndcg@5: 0.4398332209132783
dim_768_cosine_ndcg@10: 0.494222